# Homework 1 (Question 5)

For this question, you need to write a Python program that takes an input dataset and computes its pairwise correlation and kernel similarity matrix. Follow the step-by-step procedure given in this tempate. Rename the template file to hw1.ipynb and submit the Jupyter notebook along with its HTML version (by choosing File -> Download as HTML on the menu option). For this homework, you will be using the heart.csv file from lecture 1 as input data. You should not use any numpy or other library functions except for the ones explicitly mentioned in the questions below. If you're unsure whether a built-in function can be used, you can post your question on Piazza. 

### Step 1: Load the input data using pandas. 
After loading, convert the pandas DataFrame object into ndarray object named arr. Type arr.shape to verify the array size is correct.

In [287]:
import pandas as pd

data = pd.read_csv('heart.csv')
data.columns = ['age','sex','cp','bp','cholesterol','blood sugar','ecg','heart rate','angina',
               'oldpeak','slope','major vessels','thal','heart attack']
arr = data.values
print(arr.shape)

(303, 14)


### Step 2: Write a function to compute the mean and standard deviation of a 1-dimensional array.

You must not use the built-in mean or standard deviation functions to answer this question. You are allowed to use only numpy add, reduce, sqrt, and shape functions to do this. If you are not familiar with numpy's ufunc aggregate functions, you may refer to the example given in https://jakevdp.github.io/PythonDataScienceHandbook/02.03-computation-on-arrays-ufuncs.html.

**(a)** Construct your own function to compute the mean ($\mu$) and standard deviation ($\sigma$) for an array of values:
$$\mu = \frac{1}{N} \sum_{i=1}^N x_i, \ \ \ \sigma = \frac{1}{N-1} \sum_{i=1}^N (x_i - \mu)^2$$

In [288]:
import numpy as np
import math

def myMean(vec):
    col_means = np.zeros(vec.shape[1])
    for col_number in range (vec.shape[1]):
        col_means[col_number] = sum(vec[:,col_number])/len(vec[:,col_number])
    
    return col_means
def myStdDev(vec):
    avg = myMean(vec)
    col_stdevs = np.zeros(vec.shape[1])
    for col_number in range (vec.shape[1]):
        sum_of_square = sum((vec[i,col_number] - avg[col_number])**2 for i in range(vec.shape[0]))
        col_stdevs[col_number] = math.sqrt(sum_of_square/(len(vec[:,col_number])-1))
    return col_stdevs


**(b)** Create a function that returns the mean and standard deviation values for every column in a given input matrix. You may use np.zeros() to initialize the mean_values and stdDev_values arrays.

In [289]:
def calc_Mean_Stdev(arr):
    mean_values = np.zeros( arr.shape[1] )
    stdDev_values = np.zeros( arr.shape[1] )

    mean_values = myMean(arr) 
    stdDev_values = myStdDev(arr)
    
    return(mean_values, stdDev_values)
        
# To double-check your answer
(m, s) = calc_Mean_Stdev(arr)
result = pd.DataFrame(m, columns=['Mean'])
result['Std Dev'] = s
result.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
Mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
Std Dev,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835


### Step 3: Compute the correlation matrix

Create a matrix (i.e., 2-dimensional array) that contains the correlation between every pair of columns in the data. The correlation between a pair of column vectors, $\mathbf{u}$ and $\mathbf{v}$, is defined as:
$$C_{uv} = \frac{1}{N-1} \sum_{k=1}^N \frac{(u_{k} - \mu_u)(v_{k} - \mu_v)}{\sigma_u \times \sigma_v} = \frac{1}{N-1} \sum_{k=1}^N z_{ku} z_{kv},$$
where $\mathbf{z}_u = \frac{\mathbf{u} - \mu_u}{\sigma_u}$ is a vector containing the standardized values for $\mathbf{u}$, $\mathbf{z}_v = \frac{\mathbf{v} - \mu_v}{\sigma_v}$ is a vector containing the standardized values for $\mathbf{v}$, $z_{ku}$ is the k-th element of $\mathbf{z}_u$, $z_{kv}$ is the k-th element of $\mathbf{z}_v$, $\mu_u$ is the mean of vector $\mathbf{u}$, $\sigma_u$ is the standard deviation for vector $\mathbf{u}$, $\mu_v$ is the mean of vector $\mathbf{v}$, and $\sigma_v$ is the standard deviation for vector $\mathbf{v}$. 

**(a)** First, define a function to standardize the values in a 2-dimensional array. The function will subtract each element of the 2-dimensional array by its respective mean and dividing by its respective standard deviation. You can use numpy array broadcasting capabilities to do this. If you're not familiar with broadcasting, read the following tutorial (https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html) 

In [290]:
def standardize(arr):
    avg = myMean(arr)
    stdev = myStdDev(arr)
    standardized_array = arr 
    for col_number in range (arr.shape[1]):
        for i in range(arr.shape[0]):
            standardized_array[i,col_number] = (arr[i,col_number] - avg[col_number])/stdev[col_number]
    return standardized_array

# To double-check your answer
std_arr = standardize(arr)
print(std_arr)

[[ 0.95062402  0.67988052  1.96986425 ... -0.71324897 -2.14532378
   0.91301882]
 [-1.91214969  0.67988052  1.00092128 ... -0.71324897 -0.51207477
   0.91301882]
 [-1.47172297 -1.46599238  0.03197832 ... -0.71324897 -0.51207477
   0.91301882]
 ...
 [ 1.50115743  0.67988052 -0.93696465 ...  1.2425378   1.12117424
  -1.09165293]
 [ 0.28998393  0.67988052 -0.93696465 ...  0.26464441  1.12117424
  -1.09165293]
 [ 0.28998393 -1.46599238  0.03197832 ...  0.26464441 -0.51207477
  -1.09165293]]


**(b)** Create a function that computes the correlation between every pair of columns using the standardized array values returned by the function given in part (a). You should use the numpy.dot() function to compute a dot product between two vectors. You may use np.zeros() to initialize the correlation matrix.

In [291]:
def calculate_corr(arr):

    std_arr = standardize(arr)
    corr_matrix = np.zeros((arr.shape[1],arr.shape[1]))
    
    for i in range(corr_matrix.shape[1]):
        for j in range(corr_matrix.shape[1]):
            corr_matrix[i,j] = 1/((arr.shape[0])-1)*np.dot(std_arr[:,j], std_arr[:,i])
            
    return corr_matrix

# To double-check your answer
pd.DataFrame(calculate_corr(arr), columns=data.columns, index=data.columns)

,age,sex,cp,bp,cholesterol,blood sugar,ecg,heart rate,angina,oldpeak,slope,major vessels,thal,heart attack
age,1.000000,-0.098447,-0.068653,0.279351,0.213678,0.121308,-0.116211,-0.398522,0.096801,0.210013,-0.168814,0.276326,0.068001,-0.225439
sex,-0.098447,1.000000,-0.049353,-0.056769,-0.197912,0.045032,-0.058196,-0.044020,0.141664,0.096093,-0.030711,0.118261,0.210041,-0.280937
cp,-0.068653,-0.049353,1.000000,0.047608,-0.076904,0.094444,0.044421,0.295762,-0.394280,-0.149230,0.119717,-0.181053,-0.161736,0.433798
bp,0.279351,-0.056769,0.047608,1.000000,0.123174,0.177531,-0.114103,-0.046698,0.067616,0.193216,-0.121475,0.101389,0.062210,-0.144931
cholesterol,0.213678,-0.197912,-0.076904,0.123174,1.000000,0.013294,-0.151040,-0.009940,0.067023,0.053952,-0.004038,0.070511,0.098803,-0.085239
blood sugar,0.121308,0.045032,0.094444,0.177531,0.013294,1.000000,-0.084189,-0.008567,0.025665,0.005747,-0.059894,0.137979,-0.032019,-0.028046
ecg,-0.116211,-0.058196,0.044421,-0.114103,-0.151040,-0.084189,1.000000,0.044123,-0.070733,-0.058770,0.093045,-0.072042,-0.011981,0.137230
heart rate,-0.398522,-0.044020,0.295762,-0.046698,-0.009940,-0.008567,0.044123,1.000000,-0.378812,-0.344187,0.386784,-0.213177,-0.096439,0.421741
angina,0.096801,0.141664,-0.394280,0.067616,0.067023,0.025665,-0.070733,-0.378812,1.000000,0.288223,-0.257748,0.115739,0.206754,-0.436757
oldpeak,0.210013,0.096093,-0.149230,0.193216,0.053952,0.005747,-0.058770,-0.344187,0.288223,1.000000,-0.577537,0.222682,0.210244,-0.430696


### Step 4: Compute the kernel similarity matrix

For this question, you need to compute the Gaussian radial basis function (rbf) kernel similarity matrix between every pair of rows in the 2-dimensional array. Given a pair of row vectors, $\mathbf{X}_i$ and $\mathbf{X}_j$, their rbf kernel similarity can be computed as follows:
$$\mathbf{K}_{ij} = \exp\bigg[-\gamma\|\mathbf{X}_i - \mathbf{X}_j\|^2_2\bigg]$$
where $\gamma$ is a hyperparameter specified by the user.

**(a)** First, create a function that returns the squared L2-norm (Euclidean distance) between a vector and all the rows in a given matrix. **Caution:** Be careful when you perform broadcasting between a vector and a matrix. You may need to use np.newaxis to do this.

In [294]:
def sqEuclidean(vec, arr):
    m,n = arr.shape
    sqEuclidDist = np.zeros(m)
    for i in range(m):
        diff = vec - arr[i,:]
        sqEuclidDist[i] =  np.dot(diff, diff) 
        
    return sqEuclidDist

# To double-check your answer
arr = data.values
print(sqEuclidean(arr[0,:],arr))

[0.0000000e+00 2.5644400e+03 2.0458100e+03 1.4802500e+03 1.5491890e+04
 1.7616100e+03 3.8130000e+03 2.4342900e+03 2.1632400e+03 4.8704900e+03
 2.5921000e+02 2.3484100e+03 1.9648900e+03 1.7502500e+03 2.7016900e+03
 1.0604900e+03 1.2597290e+04 1.3830900e+03 1.0786400e+03 1.1025000e+02
 2.5724000e+02 1.4396100e+03 1.3202900e+03 3.0469000e+02 2.5058100e+03
 5.2136100e+03 5.3849000e+02 5.4728900e+03 3.3944250e+04 1.6282100e+03
 3.6512900e+03 3.8876100e+03 2.2412900e+03 2.0922400e+03 1.5778100e+03
 3.5398100e+03 5.6352900e+03 3.4249000e+02 1.4152500e+03 1.6369250e+04
 5.8676400e+03 1.5054100e+03 2.6474900e+03 1.3516100e+03 9.3812900e+03
 9.7094100e+03 1.3022900e+03 9.2929000e+02 1.9162900e+03 2.7129000e+02
 9.1024000e+02 5.4116100e+03 2.6325000e+02 1.0827890e+04 9.5829000e+02
 1.3442500e+03 2.1912900e+03 3.1982900e+03 4.7582900e+03 5.3282900e+03
 2.7262900e+03 7.2812900e+03 4.6662900e+03 1.8202900e+03 7.7029000e+02
 4.3748100e+03 2.3462100e+03 1.1858900e+03 1.5712900e+03 1.2092900e+03
 1.350

**(b)** Define a function that calculates the rbf kernel similarity between all pairs of rows (with hyperparameter $\gamma$=0.001). You may use numpy exp() function for this question. Your function should call the sqEuclidean() function in part (a) to return the square Euclidean distance value between a vector and a matrix.

In [295]:
def calcKernel(arr, gamma):

    m,n = arr.shape
    similarity = np.zeros((m,m))
    for i in range(m):
        similarity[i,:]=np.exp(-gamma*sqEuclidean(arr[i,:],arr))
    return similarity

# To double-check your answer
sim = calcKernel(arr, 0.001)
pd.DataFrame(sim[:10,:10])

,0,1,2,3,4,5,6,7,8,9
0,1.000000e+00,0.076962,1.292754e-01,2.275808e-01,1.870500e-07,1.717681e-01,2.208183e-02,0.087660,1.149521e-01,7.669606e-03
1,7.696227e-02,1.000000,9.362948e-02,4.722296e-01,6.720083e-06,4.513827e-03,2.838655e-02,0.587282,5.353525e-03,4.525510e-04
2,1.292754e-01,0.093629,1.000000e+00,2.497337e-01,1.088947e-10,3.389170e-01,1.527039e-04,0.027434,1.332125e-01,1.409937e-01
3,2.275808e-01,0.472230,2.497337e-01,1.000000e+00,7.135310e-07,3.915763e-02,1.237286e-02,0.406716,1.293149e-02,3.916174e-03
4,1.870500e-07,0.000007,1.088947e-10,7.135310e-07,1.000000e+00,2.133964e-12,1.648194e-02,0.000193,2.382168e-12,3.378262e-16
5,1.717681e-01,0.004514,3.389170e-01,3.915763e-02,2.133964e-12,1.000000e+00,2.939135e-05,0.001948,2.714411e-01,2.568045e-01
6,2.208183e-02,0.028387,1.527039e-04,1.237286e-02,1.648194e-02,2.939135e-05,1.000000e+00,0.147978,3.897306e-05,7.378650e-08
7,8.765996e-02,0.587282,2.743433e-02,4.067161e-01,1.927031e-04,1.947595e-03,1.479782e-01,1.000000,9.235829e-04,4.109763e-05
8,1.149521e-01,0.005354,1.332125e-01,1.293149e-02,2.382168e-12,2.714411e-01,3.897306e-05,0.000924,1.000000e+00,1.984516e-01
9,7.669606e-03,0.000453,1.409937e-01,3.916174e-03,3.378262e-16,2.568045e-01,7.378650e-08,0.000041,1.984516e-01,1.000000e+00
